In [ ]:
import pandas as pd
import numpy as np
import os

# === 1. Ler base de dados ===
CSV_PATH = "PT-FireProg_v2.1_L2_p_meteo_crg.csv"
df = pd.read_csv(CSV_PATH, sep=";")

# === 2. Selecionar apenas variáveis numéricas ===
df_num = df.select_dtypes(include=[np.number]).copy()

# === 3. Detectar automaticamente a coluna alvo 'ros p' ===
target_candidates = [c for c in df_num.columns if "ros" in c.lower() and "p" in c.lower()]
if target_candidates:
    target_col = target_candidates[0]
else:
    raise ValueError("❌ Não encontrei nenhuma coluna parecida com 'ros p' na base de dados.")

print(f"✅ Coluna alvo identificada: {target_col}")

# === 4. Função de transformação SEGURA (mantém valores válidos) ===
def transformar(series, tipo):
    series = pd.to_numeric(series, errors="coerce")
    series = series.fillna(series.median())  # substitui NA por mediana
    if tipo == "linear":
        return series
    elif tipo == "log":
        # desloca toda a série para valores positivos antes de aplicar log
        minv = series.min()
        if minv <= 0:
            series = series - minv + 1e-6
        return np.log1p(series)
    elif tipo == "sqrt":
        # garante não-negatividade
        minv = series.min()
        if minv < 0:
            series = series - minv
        return np.sqrt(series)
    else:
        raise ValueError("Transformação desconhecida.")

# === 5. Combinações pedidas ===
combinacoes = [
    ("linear", "linear"),
    ("linear", "log"),
    ("log", "log"),
    ("log", "linear"),
    ("linear", "sqrt"),
    ("sqrt", "sqrt"),
    ("sqrt", "linear"),
]

# === 6. Pasta de saída ===
os.makedirs("datasets_transformados_validados", exist_ok=True)

# === 7. Loop principal ===
for t_ros, t_outras in combinacoes:
    print(f"\n➡ Transformação: ros p ({t_ros}) vs variáveis ({t_outras})")

    # Transformar ros p
    ros_t = transformar(df_num[target_col], t_ros)
    df_transf = pd.DataFrame({target_col: ros_t})

    # Transformar restantes variáveis
    for col in df_num.columns:
        if col == target_col:
            continue
        df_transf[col] = transformar(df_num[col], t_outras)

    # Garantir que não há colunas totalmente vazias
    df_transf = df_transf.fillna(df_transf.median())

    # Calcular correlações
    corr = df_transf.corr()[target_col].sort_values(ascending=False)

    # Guardar dataset transformado
    out_name = f"datasets_transformados_validados/{t_ros}_{t_outras}.csv"
    df_transf.to_csv(out_name, index=False, sep=";")

    print(f"  → Guardado: {out_name}")
    print(f"  → Linhas: {len(df_transf)}, Colunas: {len(df_transf.columns)}")
    print("  Correlações mais fortes:")
    print(corr.head(10))
    print("  ---")

print("\n✅ Todos os datasets transformados foram gerados em 'datasets_transformados_validados/'")